# Data 301 Final Project: Team Fortress 2 Unusual Data Analysis
## By Ben Cahill

## Introduction

**Team Fortress 2** (or TF2 for short) is a first-person, class based shooter game developed by **Valve Software**. The game was originally released in 2007 and is best known for it's **cartoon graphics**, it's **nine playable classes**, and it's **in-game item economy**. In this project, I am going to take a closer look at the game's economy by analyzing data about game's **rarest and most expensive** items, **unusuals**.

![](./PresentationPictures/tf2.png) 

*The Iconic Team Fortress 2 Logo.*

During high school, I fell in love with Team Fortress 2. However, I didn't really enjoy the game because of it's first-person shooter gameplay. Instead, I loved the game because of its **virtual item economy**. You see, just like there are people buying and selling stocks in real life, there are many players buying and selling TF2 items for profit, making money by acting as middleman between those wishing to sell items those looking to buy items. This hidden trading ecosystem excited me, and after playing the game for a few months, I began to learn how to trade, eventually learning enough about the market to make **thousands of dollars** in real world money buying and selling virtual cosmetics. This love of trading also inspired me to create my own **YouTube channel** surrounding TF2's virtual economy, a channel with almost 40,000 subscribers to date. 

![](./PresentationPictures/bannersmall.PNG) 

*My YouTube channel's banner. The channel is aptly named TheVirtualEconomist.*

## The Goal of the Project

As a YouTuber, many people look up to me in the trading community, and I am often asked for trading advice. One of the most common questions I get asked is the following:

- What is the **best crate** to **"unbox"** to maximize the chance of **"unboxing"** an expensive **unusual**?

If you don't play TF2, this question probably doesn't make any sense to you. Let me give you a little bit of background.


- In Team Fortress 2, there are many different **"crates"** in the game. Each crate works like a traditional **booster pack** or **loot box**, as it **contains exactly one randomized item** that you can wear on your characters, such as a **hat** (the most iconic type of cosmetic).
- In order to **"unbox"** a crate, a player must buy a **"key"** from the in-game store. Each key costs approximately **\$2.50**, and can be used to **"unlock"** one crate before it is consumed. (As you might expect, this is how the game is monetized by Valve.)
- Most items in crates are worth less than **fifty cents**. However, you have an approximately **1%** chance to unbox an **unusual** item, an item with a **rare particle effect** surrounding it. These items can be worth anywhere from **10 dollars** to **6,000 dollars**.
- Each unusual is uniquely identified by the combination of a **cosmetic** and a **particle effect**. There are a plethora of different unusuals out there, and the **best-looking** items typically sell for the most amount of money. Once an unusual sells, it is priced at that amount on the community website **backpack.tf**, priced using the in-game currency of **keys** (the same keys you use to unbox crates). Since each key costs **$2.50**, an unusual priced at **10 keys** would be worth roughly **\$25**. An unusual priced at **100 keys** would be worth **\$250**, and so on.


![](./PresentationPictures/teamCaptain.PNG)

*On the left is a regular hat, worth 33 cents. On the right is an unusual version of the hat, worth $6,600! Notice the particle effect?*

Essentially, by asking this question, players want to know what **crate** has the best **expected value (EV)** for unboxing **unusuals**. This motivation makes sense, simply because unboxing is expensive. If you are going to drop hundreds of dollars for the **1%** chance to pull an expensive item, it should make sense that you want to give yourself the **best odds** and making your money back.

In the past, I've always been able to give people rough answers to this question based off of **domain knowledge**. However, I've never been able to **systematically** determine the correct answer **until now.** Using as much data as I can find about every unusual that exists or could possibly exist, I am going to **systematically** find the **crate** with the highest **expected value**.

Before I start wrangling the data, I think it is important to point out that that this research question is **not only** a question of **analyzing and visualizing a dataset**. As it turns out, **not every unusual that can be unboxed has been unboxed yet**, and in order to most accurately simulate the process of unboxing a **crate**, I will have to **train a machine learning model** to predict the prices of these never before seen unusuals.


## Collecting & Cleaning the Data

Since I couldn't find any csv files containing the data I needed, I had to manually compile everything myself. This involved scraping many pages from the **Team Fortress 2 Wiki**, using a **JSON API** to download unusual prices (from the website Backpack.tf), and creating a **Google survey** to get additional data from experienced traders.


**In total, the following data was pulled, compiled, and cleaned.**

- List of all possible unusual cosmetics scraped from the TF2 Wiki [found here](https://wiki.teamfortress.com/wiki/Template:Unusual_quality_table).
- List of all possible unusual effects scraped from the TF2 Wiki [found here](https://wiki.teamfortress.com/wiki/Unusual).
- List of unusual effect IDs scraped from Backpack.tf [found here](https://backpack.tf/developer/particles). *Note:* These ID numbers are only needed to fetch pricing data from Backpack.tf and are not used in any machine learning models.  
- List of which unusual cosmetics can be "unboxed" out of each crate, scraped from a community-created guide, [found here](https://steamcommunity.com/sharedfiles/filedetails/?id=731640447). 
- List of additional descriptive data from each unusual, compiled by scraping each cosmetic's wiki page, [found here](https://wiki.teamfortress.com/wiki/).
- List of which cosmetics are "robo" hats (robotic versions of normal hats), scraped from the TF2 Wiki [found here](https://wiki.teamfortress.com/wiki/Robotic_Boogaloo).
- List of effect ratings (on a scale of 0 to 5), compiled via a Google Form sent out to experienced traders. The forum can be [found here](https://docs.google.com/forms/d/e/1FAIpQLSd2kf5WsAKLIQfwrujgNKBFBhWeS0_ukJkfob3hzJ4-kw7XAA/viewform).
- Prices for each unusual from Backpack.tf's JSON API, [found here](https://backpack.tf/developer).


Once I had downloaded all of the data, I merged everything into a single master dataset. Since this dataset included **all possible combinations of unusuals** (even if they didn't exist) the dataset was **massive**, with over 41,000 observations!

![](./PresentationPictures/DatasetAtThisPoint.PNG) 

## Visualizing the Data

Because I had a lot of **categorical variables**, I found **Altair** plots to be a significant asset in visualizing my data, as I could display variables via the size of points, color of points, or in different facets. 

![](./PresentationPictures/altairplot1.PNG)

In TF2 there are two main types of unusuals, unusual **hats (cosmetics)** and unusual **taunts**. This first plot displays the differences between the two types. It is immediately noticeable that:

1. There are far **more** hats than taunts.
2. **Taunts are very inexpensive**. No hat is worth less than around 8 keys, yet there are a significant number of taunts worth as little as 4 keys.

![](./PresentationPictures/taunt.jpg)

*An example of an unusual taunt.*

Since the points are colored based on the **class** (the playable character) the unusuals are for, you can also notice that unusuals that **can be worn on more than one class** (denoted in **orange** and **yellow**) are worth, on average, **much more** than unusuals that can only be worn on **one class** (denoted in **blue**).


Both this graph and the graph below also show that there is a rough **correlation** between the price of an unusual and how high the community rated it's effect. The higher the community rated an unusual effect, the more it tends to be priced at. This makes quite a bit of sense, since the market is driven by **supply and demand**. If more people like the look of an unusual, it is probably going to sell for more! 

![](./PresentationPictures/altairplot2.PNG)

Just as there are more unusual **hats** than **taunts**, there are more different types of **hats** than **taunts**. The plot above makes three distinctions in terms of types of hats. On the left are **Robotic** (**"Robo"** for short) hats. **Robo** hats are **robotic versions of other hats already in the game**, added during a game update themed around robots. In general the community thinks that these hats are **ugly** compared to their normal counterparts, and this disdain is visualized through the **lower prices** of **robo** hats in the graph above.

On the other hand, unusual **miscs** (shown on the right) operate in an almost opposite fashion. Unusual **miscs** are unusuals that, due to their shape and size, **can we worn at the same time as another unusual hat**. Due to the fact that players can combine different unusual effects to create flashy particle displays, unusual **miscs** are in very high demand, something that can be visualized through the **higher prices** of hats in the graph above.

In 2015, Valve added an additional rarity system to crate-based items. Instead of every item having the same underlying rarity, there became four distinct **"grades"** of item, **Mercenary**, **Commando**, **Assassin** and **Elite**. When unboxing a crate, you have about an **80%** chance of getting a **Mercenary** item, a **15%** chance of getting a **Commando** item, a **3%** chance of getting an **Assassin** item, and a **0.6%** chance of getting an **elite** item. When you remember that you have a **1%** chance of unboxing an unusual in the first place, it follows that the chance to pull an **unusual elite grade** item is 0.01 * 0.006 or as low as **0.006%!** As such, there are barely any **elite** items in existence (and subsequently in the dataset). The plot above reflects these rarities in the price of each item, as **Mercenary** grade items tend to be fairly cheap compared to **Assassin** and **Elite** grade items. 

![](./PresentationPictures/grades.PNG)

*The four grades*

Interestingly enough, you can also tell from the scatterplot above that the Robotic Boogaloo (**robo**) update occurred *before* 2015, since there are no graded robo items. You can also tell that Valve has added few miscs to the game since 2015, as there are only a few **Commando** grade miscs and no miscs of any other grade.


![](./PresentationPictures/halloweenplot.PNG)

Each October, Valve releases a Halloween update for TF2, typically adding brand new **halloween-themed unusual effects** into the game. These new effects can **only be unboxed during the Halloween season**, and are much rarer than regular unusual effects. As shown by the plot above, this higher rarity appears to make halloween effects **more expensive** than their regular counterparts. It also appears that the community favors these effects from an aesthetic standpoint, **as most Halloween effects have higher community ratings**. Again, price and community rating are correlated.

![](./PresentationPictures/bestcratesfirst.PNG)

While the previous plots are great at visualizing the data, they don't actually help answer the research question. 

Since there is a variable in the dataset (`Unboxed From`) that denotes what crate each cosmetic is from, can't we just answer the research question by **grouping the data by crate** and then taking the **mean unusual price**? (As shown in the plot above)

Unfortunately, this **doesn't quite work**, for a number of reasons.

1. This **doesn't accurately simulate the unboxing process**. When unboxing a crate, you get an unusual based on **random chance**. Taking the mean of each crate would assume that you pulled exactly one of each type of unusual in your sample set, which isn't realistic. Additionally, this doesn't factor in the added rarities of **graded** items.
2. Many types of unusuals have weird properties when unboxing, which cannot be represented here. For example, **graded** unusuals are unboxed from **crates** that also have an additional chance of unboxing an **unusual taunt**, which isn't reflected in the plot.
3. This model only uses unusuals that **already exist** and **have a price on backpack.tf**. As mentioned before, there are many unusuals that can be unboxed that don't exist yet. A good model must take this into consideration.

The solution to these problems is a two step process.

1. Develop a **machine learning model** to **predict the price of every unusual that doesn't exist yet**.
2. Create a **realistic unboxing simulator** that uses both predicted and actual data to generate **randomized unusual unboxes**.

Once these two steps have been accomplished, you can simply revisualize the data to obtain obtain the correct results!

## Machine Learning

One thing I noticed about my dataset was that it is almost **entirely categorical**. While I knew `Price` and `Effect Community Rating` were going to be crucial for machine learning, I realized quickly that `Months Since Last Price Update` had no correlation to anything else in the dataset, leaving the dataset filled with 2 quantitative variables and almost 10 categorical variables. 

Unfortunately, this large amount of categorical variables turned out to significantly hamper the performance of any K-Nearest Neighbors models I ran on the data. This was because there were some rare cosmetics in the dataset (typically **elite grade items**) with only a few priced unusuals, causing KKN to **fail to find enough reasonable neighbors**.

To solve this issue, I created two new quantitative metrics, `Median Hat Rating` and `Median Effect Rating` using a combination of existing variables. These variables essencially gave me a tier order distinguishing the **best cosmetics** from the **worst cosmetics**, and the **best effects** from the **worst effects**

![](./PresentationPictures/medianHatRating.PNG)
![](./PresentationPictures/medianEffectRating2.PNG)

Since my dataset included unusuals **with and without a price**, my first step was to split the dataset into a two sub-sets, a **training** and **prediction** set, based on whether each unusual had a price.

Using the visualizations I created before, I selected a feature set from variables that **seemed reasonably correlated with price**. Next, I tried many different machine learning models on the dataset, ultimately finding that **KKN**, **RadomForestRegressor** and **Bagging Regressor** yielded some of the lowest cross-validation scores. I also used hyperparameter testing to determine the optimal value of **k in KKN** and **n (number of estimators) in RandomForestRegressor**.

![](./PresentationPictures/KNN.PNG)
![](./PresentationPictures/Forest.PNG)

*Left: Testing for k. Right: Testing for n_estimators.*

In general, the cross-validation scores were **much higher than I expected**, averaging about 50 keys. However, this **doesn't necessarily mean that my predictions were bad**. Due to the distribution of unusuals in the training dataset, there are some **iconic** or **glitched unusuals** with almost unpredictably high prices (e.g. no other similar unusuals were priced that high), causing my model to always under-predict these items. Additionally, I know from domain knowledge that **the prices of expensive unusuals (>500 keys) are often less concrete than the prices of cheaper unusuals (<500 keys)**. This is because far **fewer people can afford to buy expensive items**, and **it is more important to see an unusual sell for anything than to see it sell for the "exact" Backpack.tf price**. Thus, even though my model is quite a bit less accurate at predicting expensive items, it doesn't really matter if, for example, the model predicts an unusual to be worth 2,100 keys instead of 1,900 keys, since as long as the model can determine the rough magnitude of price (e.g. 2000 keys instead of 200 keys), the model is a good predictor.

To make my final predictions, I used a **Linear Regression Regression Ensembler** to combine my **KKN**, **RadomForestRegressor** and **Bagging Regressor** models, using a straight average. This yielded good predictions, since **KNN tended to underpredict prices**, and **RandomForestRegressor & Bagging Regressor tended to overpredict prices**, giving me a good mean prediction.


## Probabilities (& More Visualizations) 

Once I obtained predictions for each unpriced unusual, I merged the predicted prices back with the training data to obtain a dataset with prices for every unusual. Using domain knowledge (that took into account the many intricacies of unboxing), I then generated a **single function**, called `simulate_unboxing`, that could, given any **crate**, predict the average price of an unusual unbox out of that **crate**. By default, the function simulates 1,000,000 "unboxes", but that can be customized with a different parameter if necessary. 


Calling the function yields the following results. The X-axis represents the average price of an unusual unboxed from each crate.

![](./PresentationPictures/resultsRegular.PNG)

As it turned out, the **most profitable crate** ended up being the **Multi/All Class Crate**. This wasn't super surprising to me, since the original data analysis already revealed that **multi-class** cosmetics are typically **worth more** than **single class** cosmetics. Since the **Multi/All Class Crate** contains **only** multi-class cosmetics, it goes to figure that the average unusual price in that crate would be **high**. That being said, the the 2nd and 3rd place crates are more surprising. As opposed to most of the other high-scoring crates, both the **End of the Line** and **Robotic Boogaloo** crates are **relatively old** (being released in 2013 and 2014 respectively), causing them to be less well known by modern players. Thus, despite the high unusual EV, I've never see anyone give the advice to unbox these crates. If I end up publishing these results to the community (e.g by means of a video), this is going to be big news to those looking to unbox.

However, even if there are some crates with high unusual EV, is it necessarily **worth it** to unbox? That is, **in the long run**, as the sample size increases, will you **make more money from the unusuals you unbox than you spent on keys**? My `simulate_unboxing` function returned another metric to help answer this, the **average money made or lost per key spent**. 

![](./PresentationPictures/resultsRegular2.PNG)

Unfortunately (as shown above), the results are not promising. In the long run, **every crate** is unprofitable **except** the **Multi-Class crate**, which appears to **breaks even**, meaning you get back **as much money** in **unusuals** as you spent on **keys**. Thus, while unboxing could yield net **positive results in the short run** (think of someone who opens **100 crates** and unboxes a **200 key unusual**), as the sample size increases you are **guaranteed to either break even or lose money**.

However, as I mentioned earlier, every October the **regular effects are swapped out for Halloween effects**. Further, the data analysis showed that **Halloween effects** are worth **much more** than **regular effects**. **Does this mean that unboxing can be profitable during Halloween**? 

I ran my `simulate_unboxing` function again, this time passing the parameter `halloween = true`. The first plot displays the **average unusual unbox price**, and the second plot displays the **average money made or lost per key spent**, like before.

![](./PresentationPictures/HalloweenUnboxes.PNG)

![](./PresentationPictures/HalloweenUnboxes2.PNG)

The results here are fascinating. Given the right **crate**, unboxing during Halloween **is profitable in the long run!** In fact, if you unbox only **Gunn Mettle Cosmetic Case**, you can net as much as 3 dollars in profit per each crate you unbox. Interestingly, the **Gunn Mettle Cosmetic Case** appears to be so profitable because of its unique distribution of **graded** cosmetics. Within the crate, the only **mercenary grade hat** is a aesthetic-looking **all-class hat**, and since **mercenary grade** items drop about **80%** of the time, it is very likely for unboxers to pull this **all-class hat** with a fantastic Halloween effect.

You may notice that the **Multi-Class crate**, as well as the **2nd and 3rd place crates from before**, are not anywhere close to the best performers during Halloween. This is because Valve recently limited Halloween effects soley to new **graded crates**, meaning that during Halloween the **Multi-Class crate** still drops regular effects.

While unboxing during Halloween might seem very profitable right now, keep in mind that **EV cannot stay high forever**. People are drawn to unbox profitable crates, causing **the supply of unusuals from that crate to increase, thus lowering prices and EV**. If people were **perfectly rational** in terms of their unboxing and item pricing habits, we should expect **every** crate to be unprofitable after existing for a few years.

## Conclusion

All in all, the following conclusions can be drawn.

1. Unboxing during the **regular season** is **not profitable**. While you could come out ahead by unboxing a small number crates and getting lucky, you are **systematically guaranteed to either break even or lose money in the long run**.
2. If you unbox a select few crates (such as the **Gunn Mettle Cosmetic Case**) you can make massive amounts of money unboxing **during the Halloween season**. Just keep in mind that these high prices may not last forever.

## Questions/Comments?